In [1]:
import pyspark

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, avg, max, desc, when, expr,round

spark = SparkSession.builder.appName("CovidAnalysis").getOrCreate()

df1 = spark.read.csv(r'C:\Users\Leon\Downloads\covid\data\worldometer_data.csv', header=True, inferSchema=True)
df2= spark.read.csv(r'C:\Users\Leon\Downloads\covid\data\covid_vaccine.csv', header=True, inferSchema=True)
spark


1. To find out the population vs the number of people vaccinated


In [17]:
df_joined = df1.join(df2,df1["Country/Region"]==df2["Country"],"inner")
df_joined=df_joined.groupBy(col("Country")).agg(
    sum("Population").alias("Total_Population"),
    sum("people_fully_vaccinated").alias("Total_Fully_Vaccinated")
)

df_joined.select(col("Country"),col("Total_Population"),col("Total_Fully_Vaccinated")).show()


+-------+----------------+----------------------+
|Country|Total_Population|Total_Fully_Vaccinated|
+-------+----------------+----------------------+
| Russia|       145940924|              25000000|
|Germany|       167622520|              60000000|
|  India|      2762689994|             130000000|
|    USA|       993594390|             360000000|
|     UK|       135844058|              50000000|
| Brazil|       212710692|              30000000|
+-------+----------------+----------------------+



2.  To find out the percentage of different vaccine taken by people in a country

In [26]:
df_joined = df1.join(df2,df1["Country/Region"]==df2["Country"],"inner")
df_joined= df_joined.withColumn("Percentage of vaccine",round((col("people_vaccinated") / col("Population")) * 100, 2))
df_joined.select(col("Country/Region"),col("vaccine"),col("Percentage of vaccine")).show()


+--------------+------------------+---------------------+
|Country/Region|           vaccine|Percentage of vaccine|
+--------------+------------------+---------------------+
|           USA|   Johnson&Johnson|                54.35|
|           USA|           Moderna|                54.35|
|           USA|   Pfizer/BioNTech|                54.35|
|        Brazil|Oxford/AstraZeneca|                32.91|
|         India|        Covishield|                10.86|
|         India|           Covaxin|                10.86|
|        Russia|         Sputnik V|                27.41|
|            UK|           Moderna|                88.34|
|            UK|Oxford/AstraZeneca|                88.34|
|       Germany|           Moderna|                71.59|
|       Germany|          BioNTech|                71.59|
+--------------+------------------+---------------------+

